# %PPPH Comparison between sPV/PV and P'V/PV
Against ori

In [61]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# from C_0B_eval import *
from C_0X_defs import *
from C_0Y_evaldefs import *
from C_0Z_evaldefsAppendix import *
from scipy.stats import sem, ttest_ind
from misc_recorder import ListRecorder
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
import re

In [62]:
spvpv_ts = "0910233758" # both are cosine trained and and 2-layed. 
pvpv_ts = "0911151319"

spvpv_res_save_dir = os.path.join(model_save_, f"eval-C_0Tf-{spvpv_ts}")
pvpv_res_save_dir = os.path.join(model_save_, f"eval-C_0Te-{pvpv_ts}")

In [63]:
super_data_collector = {}   # 存放所有数据
look_for_layer_path = "abx-pph-0903-2"

In [64]:
for target_field in ["spvpv;ptk", "pvpv;ptk", "spvpv;asp", "pvpv;asp"]: 
    target, field = target_field.split(";")
    goodfield = "PPP" if field == "ptk" else "PPH"
    good_target_field = target + ";" +goodfield
    numeric = "04" if field == "ptk" else "05"
    res_save_dir = spvpv_res_save_dir if target == "spvpv" else pvpv_res_save_dir
    super_data_collector[good_target_field] = {}
    for model_type in ['recon4-phi', 'recon8-phi', 'recon16-phi', 'recon32-phi', 'recon48-phi', 'recon64-phi']: 
        hiddim = int(model_type.split('-')[0].split('recon')[1])
        layered_res = {}
        for layer in ["hidrep", "attnout", "dec-lin1", "enc-lin1", 
                        "dec-rnn1-f", "enc-rnn1-f", "enc-rnn1-b",
                        "dec-rnn2-f", "enc-rnn2-f", "enc-rnn2-b"]: 
                        # "dec-rnn3-f", "enc-rnn3-f", "enc-rnn3-b", 
                        # "dec-rnn4-f", "enc-rnn4-f", "enc-rnn4-b", 
                        # "dec-rnn5-f", "enc-rnn5-f", "enc-rnn5-b", ]: # "enc-lin1", 
            asp_list_epochs = []
            layer_path = os.path.join(res_save_dir, look_for_layer_path, 
                                        f"{numeric}-save-{field}-{model_type}-b-12345-{layer}.npy")
            layer_res = np.load(layer_path)
            layered_res[layer] = layer_res.transpose(1, 0)
        # deal with ori
        ori_path = os.path.join(res_save_dir, look_for_layer_path, f"{numeric}-save-{field}-recon64-phi-b-12345-ori.npy")
        if os.path.exists(ori_path): 
            ori_res = np.load(ori_path)
            layered_res["ori"] = ori_res.transpose(1, 0)
        else: 
            raise ValueError("No ori path found.")
        
        super_data_collector[good_target_field][hiddim] = layered_res

In [65]:
# for hiddim in [4, 8, 16, 32, 48, 64, 96]: 
#     attnres = super_data_collector[hiddim]["attnres"]
#     for condition in ["T", "ST"]: 
#         conditionres = attnres[condition]
#         attnres[condition] = conditionres.transpose(1, 0, 2)

In [22]:
super_data_collector["spvpv-ptk"][4]["hidrep"].shape

(101, 30)

## Plot

In [35]:
def plot_many_plotly(arrs, labels, save_path, plot_label_dict={"xlabel": "Epoch", "ylabel": "Value", "title": "Value Across Epochs"}, y_range=None, cloud=True, steplist=None):
    """
    steplist: list of n_steps
    """
    n_steps = arrs[0].shape[1]
    mean_trajs = []
    lower_bounds = []
    upper_bounds = []
    colors = px.colors.qualitative.Alphabet
    lines = ['solid', 'dot', 'dash', 'longdash', 'dashdot', 'longdashdot']
    marks = ['circle', 'square', 'diamond', 'cross', 'x', 'triangle-up', 'triangle-down', 'triangle-left', 'triangle-right', 'pentagon', 'hexagon', 'octagon', 'star', 'hexagram', 'star-triangle-up', 'star-triangle-down', 'star-square', 'star-diamond', 'diamond-tall', 'diamond-wide', 'hourglass', 'bowtie', 'circle-cross', 'circle-x', 'square-cross', 'square-x', 'diamond-cross', 'diamond-x', 'cross-thin', 'x-thin', 'asterisk', 'hash', 'y-up', 'y-down', 'y-left', 'y-right', 'line-ew', 'line-ns', 'line-ne', 'line-nw']


    for arr in arrs:
        assert arr.shape[1] == n_steps
        mean_traj = np.mean(arr, axis=0)
        sem_arr = stats.sem(arr, axis=0)
        ci_95 = 1.96 * sem_arr
        upper_bound = mean_traj + ci_95
        lower_bound = mean_traj - ci_95
        mean_trajs.append(mean_traj)
        lower_bounds.append(lower_bound)
        upper_bounds.append(upper_bound)
    
    if steplist is not None: 
        steps = steplist
    else: 
        steps = list(range(n_steps))

    # Create Plotly traces
    fig = go.Figure()
    lidx, cidx, midx = 0, 1, 2
    labelsL = {label: idx for idx, label in enumerate(list(set([label.split(":")[lidx] for label in labels])))}
    labelsC = {label: idx for idx, label in enumerate(list(set([label.split(":")[cidx] for label in labels])))}
    labelsM = {label: idx for idx, label in enumerate(list(set([label.split(":")[midx] for label in labels])))}
    for idx, (mean_traj, lower_bound, upper_bound, label) in enumerate(zip(mean_trajs, lower_bounds, upper_bounds, labels)):
        # Add mean line
        fig.add_trace(go.Scatter(
            x=steps,
            y=mean_traj,
            mode="lines+markers", 
            name=label,
            line=dict(color=colors[labelsC[label.split(":")[cidx]]], dash=lines[labelsL[label.split(":")[lidx]]]),
            marker=dict(symbol=marks[labelsM[label.split(":")[midx]]])
        ))
        # Add confidence interval shading if cloud is True
        if cloud:
            fig.add_trace(go.Scatter(
                x=list(range(n_steps)) + list(range(n_steps))[::-1],
                y=np.concatenate([upper_bound, lower_bound[::-1]]),
                fill='toself',
                fillcolor=colors[idx],
                line=dict(color='rgba(255,255,255,0)'),
                showlegend=False,
                opacity=0.2,
            ))

    # Update layout
    fig.update_layout(
        title=plot_label_dict["title"],
        xaxis_title=plot_label_dict["xlabel"],
        yaxis_title=plot_label_dict["ylabel"],
        yaxis=dict(range=y_range if y_range is not None else [None, None]),
        legend_title="Trajectories",
        template="plotly_white"
    )
    
    # Save to HTML file
    fig.write_html(save_path)

In [175]:
def plot_many_plotly_meaned(arrs, labels, save_path, plot_label_dict={"xlabel": "Epoch", "ylabel": "Value", "title": "Value Across Epochs"}, y_range=None, cloud=True, steplist=None):
    """
    steplist: list of n_steps
    """
    n_steps = arrs[0].shape[0]
    mean_trajs = []
    colors = px.colors.qualitative.Alphabet
    lines = ['solid', 'dot', 'dash', 'longdash', 'dashdot', 'longdashdot']
    marks = ['circle', 'square', 'diamond', 'cross', 'x', 'triangle-up', 'triangle-down', 'triangle-left', 'triangle-right', 'pentagon', 'hexagon', 'octagon', 'star', 'hexagram', 'star-triangle-up', 'star-triangle-down', 'star-square', 'star-diamond', 'diamond-tall', 'diamond-wide', 'hourglass', 'bowtie', 'circle-cross', 'circle-x', 'square-cross', 'square-x', 'diamond-cross', 'diamond-x', 'cross-thin', 'x-thin', 'asterisk', 'hash', 'y-up', 'y-down', 'y-left', 'y-right', 'line-ew', 'line-ns', 'line-ne', 'line-nw']


    for arr in arrs:
        assert arr.shape[0] == n_steps
        mean_trajs.append(arr)
    
    if steplist is not None: 
        steps = steplist
    else: 
        steps = list(range(n_steps))

    # Create Plotly traces
    fig = go.Figure()
    lidx, cidx, midx = 0, 2, 1
    labelsL = {label: idx for idx, label in enumerate(list(set([label.split(":")[lidx] for label in labels])))}
    labelsC = {label: idx for idx, label in enumerate(list(set([label.split(":")[cidx] for label in labels])))}
    labelsM = {label: idx for idx, label in enumerate(list(set([label.split(":")[midx] for label in labels])))}
    for idx, (mean_traj, label) in enumerate(zip(mean_trajs, labels)):
        # Add mean line
        fig.add_trace(go.Scatter(
            x=steps,
            y=mean_traj,
            mode="lines+markers", 
            name=label,
            line=dict(color=colors[labelsC[label.split(":")[cidx]]], dash=lines[labelsL[label.split(":")[lidx]]]),
            marker=dict(symbol=marks[labelsM[label.split(":")[midx]]])
        ))

    # Update layout
    fig.update_layout(
        title=plot_label_dict["title"],
        xaxis_title=plot_label_dict["xlabel"],
        yaxis_title=plot_label_dict["ylabel"],
        yaxis=dict(range=y_range if y_range is not None else [None, None]),
        legend_title="Trajectories",
        template="plotly_white"
    )
    
    # Save to HTML file
    fig.write_html(save_path)

In [101]:
import select
import plotly.graph_objects as go

# Sample data for plotting
x = list(range(10))
traces = []

# Parameters A, B, and C with their values
param_A = ['a1', 'a2']
param_B = ['b1', 'b2', 'b3']
param_C = ['c1', 'c2']

# Generate traces for each combination of A, B, C
for a in param_A:
    for b in param_B:
        for c in param_C:
            traces.append(go.Scatter(x=x, y=[i*(1+len(traces)) for i in x], mode='lines',
                                     name=f'A: {a}, B: {b}, C: {c}',
                                     visible=True))

global fieldsel
fieldsel = [None, None, None]
# Create the figure and add traces
fig = go.Figure(data=traces)

# Initialize the visibility of traces
def get_visibility(selected_A, selected_B, selected_C):
    return [
        (trace.name.split(', ')[0] == f'A: {selected_A}' if selected_A else True) and
        (trace.name.split(', ')[1] == f'B: {selected_B}' if selected_B else True) and
        (trace.name.split(', ')[2] == f'C: {selected_C}' if selected_C else True)
        for trace in traces
    ]

def getset_visibility(fieldtosel): 
    for idx, field in enumerate(fieldtosel): 
        if field: 
            fieldsel[idx] = field

    selected_A, selected_B, selected_C = fieldsel
    return [
        (trace.name.split(', ')[0] == f'A: {selected_A}' if selected_A else True) and
        (trace.name.split(', ')[1] == f'B: {selected_B}' if selected_B else True) and
        (trace.name.split(', ')[2] == f'C: {selected_C}' if selected_C else True)
        for trace in traces
    ]

# Create dropdown menus
dropdown_A = dict(
    buttons=[dict(label='All', method='update', args=[{'visible': get_visibility(None, None, None)}])] +
            [dict(label=a, method='update',
                  args=[{'visible': getset_visibility([a, None, None])}])
             for a in param_A],
    type="dropdown", 
    direction="down",
    showactive=True,
    x=0.1,
    xanchor="left",
    y=1.15,
    yanchor="top"
)

dropdown_B = dict(
    buttons=[dict(label='All', method='update', args=[{'visible': get_visibility(None, None, None)}])] +
            [dict(label=b, method='update',
                  args=[{'visible': getset_visibility([None, b, None])}])
             for b in param_B],
    type="dropdown", 
    direction="down",
    showactive=True,
    x=0.3,
    xanchor="left",
    y=1.15,
    yanchor="top"
)

dropdown_C = dict(
    buttons=[dict(label='All', method='update', args=[{'visible': getset_visibility([None, None, None])}])] +
            [dict(label=c, method='update',
                  args=[{'visible': getset_visibility([None, None, c])}])
             for c in param_C],
    type="dropdown", 
    direction="down",
    showactive=True,
    x=0.5,
    xanchor="left",
    y=1.15,
    yanchor="top"
)

# Add dropdown menus to the figure
fig.update_layout(
    updatemenus=[
        dropdown_A,
        dropdown_B,
        dropdown_C
    ],
    title="Selectable Legends by Combination of Parameters",
    showlegend=True
)


fig.write_html(os.path.join(comp_plots_dir, f"trial.html"))


In [154]:
def plot_many_plotly_meaned_menu(arrs, labels, save_path, plot_label_dict={"xlabel": "Epoch", "ylabel": "Value", "title": "Value Across Epochs"}, y_range=None, cloud=True, steplist=None, fieldlist=None):
    """
    steplist: list of n_steps
    """
    n_steps = arrs[0].shape[0]
    mean_trajs = []
    colors = px.colors.qualitative.Alphabet
    lines = ['solid', 'dot', 'dash', 'longdash', 'dashdot', 'longdashdot']
    marks = ['circle', 'square', 'diamond', 'cross', 'x', 'triangle-up', 'triangle-down', 'triangle-left', 'triangle-right', 'pentagon', 'hexagon', 'octagon', 'star', 'hexagram', 'star-triangle-up', 'star-triangle-down', 'star-square', 'star-diamond', 'diamond-tall', 'diamond-wide', 'hourglass', 'bowtie', 'circle-cross', 'circle-x', 'square-cross', 'square-x', 'diamond-cross', 'diamond-x', 'cross-thin', 'x-thin', 'asterisk', 'hash', 'y-up', 'y-down', 'y-left', 'y-right', 'line-ew', 'line-ns', 'line-ne', 'line-nw']


    for arr in arrs:
        assert arr.shape[0] == n_steps
        mean_trajs.append(arr)
    
    if steplist is not None: 
        steps = steplist
    else: 
        steps = list(range(n_steps))

    # Create Plotly traces
    traces = []
    lidx, cidx, midx = 0, 1, 2
    labelsL = {label: idx for idx, label in enumerate(list(set([label.split(":")[lidx] for label in labels])))}
    labelsC = {label: idx for idx, label in enumerate(list(set([label.split(":")[cidx] for label in labels])))}
    labelsM = {label: idx for idx, label in enumerate(list(set([label.split(":")[midx] for label in labels])))}
    for idx, (mean_traj, label) in enumerate(zip(mean_trajs, labels)):
        # Add mean line
        traces.append(go.Scatter(
            x=steps,
            y=mean_traj,
            mode="lines+markers", 
            name=label,
            line=dict(color=colors[labelsC[label.split(":")[cidx]]], dash=lines[labelsL[label.split(":")[lidx]]]),
            marker=dict(symbol=marks[labelsM[label.split(":")[midx]]]), 
            visible=True
        ))

    fig = go.Figure(data=traces)

    fieldlist = fieldlist if fieldlist is not None else range(len(re.split(r'[:;]', labels[0])))
    fieldedlabels = {field: list(set([re.split(r'[:;]', label)[fidx] for label in labels])) for fidx, field in enumerate(fieldlist)}

    checkbox_list = []
    for fidx, field in enumerate(fieldlist): 
        buttonlist = []
        all_signal_setter = [-1 if idx == fidx else None for idx in range(len(fieldlist))]
        buttonlist.append(dict(label=f'{field}:All', method='restyle', args=[{'visible': getset_visibility(all_signal_setter)}]))
        for fieldlabel in fieldedlabels[field]: 
            field_signal_setter = [fieldlabel if idx == fidx else None for idx in range(len(fieldlist))]
            buttonlist.append(dict(label=f"{field}:{fieldlabel}", method='restyle', args=[{'visible': getset_visibility(field_signal_setter)}]))
        checkbox_dict = dict(
            buttons=buttonlist,
            type="dropdown", 
            direction="down",
            showactive=True,
            x=0.1 + 0.2 * fidx,
            xanchor="left",
            y=1.15,
            yanchor="top"
        )
        # checkbox_dict = dict(
        #     buttons=[dict(label=f"{field}:{label}",
        #                 method="restyle",
        #                 args=[{"visible": [f"{label}" in trace.name for trace in traces]}],
        #                 args2=[{"visible": [False for _ in traces]}])
        #             for label in fieldedlabels[field]],
        #     direction="down",
        #     type="buttons",  # Use buttons for checkboxes
        #     showactive=True,
        #     x=0.1 + 0.2 * fidx,
        #     xanchor="left",
        #     y=1.15,
        #     yanchor="top"
        # )
        # checkbox_dict = dict(
        #     buttons=[dict(label=f"{field}: {label}",
        #                 method="restyle",
        #                 args=[{"visible": [(f"{label}" in trace.name)  
        #                                     for trace in traces]}],
        #                 args2=[{"visible": [not (f"{label}" in trace.name) 
        #                                     for trace in traces]}])
        #             for label in fieldedlabels[field]],
        #     direction="down",
        #     type="buttons",
        #     showactive=True,
        #     x=0.1 + 0.2 * fidx,
        #     xanchor="left",
        #     y=1.15,
        #     yanchor="top"
        # )
        checkbox_list.append(checkbox_dict)

    # Update layout
    fig.update_layout(
        title=plot_label_dict["title"],
        xaxis_title=plot_label_dict["xlabel"],
        yaxis_title=plot_label_dict["ylabel"],
        yaxis=dict(range=y_range if y_range is not None else [None, None]),
        legend_title="Trajectories",
        template="plotly_white"
    )

    fig.update_layout(
        updatemenus=checkbox_list,
        showlegend=True
    )
    # Save to HTML file
    fig.write_html(save_path)

In [189]:
data_list = []
tag_list = []
for hiddim in [4, 8, 16, 32, 48, 64]: 
    for target_field in ["spvpv;PPP", "pvpv;PPP", "spvpv;PPH", "pvpv;PPH"]: 
        target, field = target_field.split(";")
        hidrepres = super_data_collector[target_field][hiddim]["enc-rnn2-f"].mean(axis=1)
        attnoutres = super_data_collector[target_field][hiddim]["attnout"].mean(axis=1)
        orires = super_data_collector[target_field][hiddim]["ori"].mean(axis=1)
        # data_list.append(hidrepres/attnoutres)
        data_list.append(hidrepres/orires)
        data_list.append(attnoutres/orires)
        # data_list.append(hidrepres)
        # data_list.append(attnoutres)
        tag_list.append(f"{target}:{field};encrnn2f:{hiddim}")
        tag_list.append(f"{target}:{field};attnout:{hiddim}")

In [187]:
comp_plots_dir = os.path.join(model_save_, "all-comp-plots")
mk(comp_plots_dir)

In [190]:
plot_many_plotly_meaned(data_list, tag_list, os.path.join(comp_plots_dir, f"encrnn2f-attnout|ori.html"), 
                plot_label_dict={"xlabel": "Epoch", "ylabel": "Scores Value", "title": f"Comparison Trajectories of PSAS and Layer Outputs"}, 
                cloud=False)

In [164]:
def getset_visibility(field_select_signal_setter): 
    for idx, field_signal in enumerate(field_select_signal_setter): 
        if field_signal is not None and field_signal != -1: 
            field_select_signal[idx] = field_signal
        elif field_signal == -1: 
            field_select_signal[idx] = None
        # else do not change
    return [
        all(
            [(re.split(r'[:;]', trace.name)[fsidx] == field_signal if field_signal else True) for fsidx, field_signal in enumerate(field_select_signal)]
        ) for trace in traces
    ]
fieldlist=["Cond", "Eval", "Rep", "Dim"]
global field_select_signal
field_select_signal = [None for _ in range(len(fieldlist))]

arrs = data_list
labels = tag_list
save_path = os.path.join(comp_plots_dir, f"hidrep-attnout-ori.html")
plot_label_dict = {"xlabel": "Epoch", "ylabel": "Scores Value", "title": f"Comparison Trajectories of PSAS and Layer Outputs"}
fieldlist = ["Cond", "Eval", "Rep", "Dim"]
steplist = None
y_range = None

n_steps = arrs[0].shape[0]
mean_trajs = []
colors = px.colors.qualitative.Alphabet
lines = ['solid', 'dot', 'dash', 'longdash', 'dashdot', 'longdashdot']
marks = ['circle', 'square', 'diamond', 'cross', 'x', 'triangle-up', 'triangle-down', 'triangle-left', 'triangle-right', 'pentagon', 'hexagon', 'octagon', 'star', 'hexagram', 'star-triangle-up', 'star-triangle-down', 'star-square', 'star-diamond', 'diamond-tall', 'diamond-wide', 'hourglass', 'bowtie', 'circle-cross', 'circle-x', 'square-cross', 'square-x', 'diamond-cross', 'diamond-x', 'cross-thin', 'x-thin', 'asterisk', 'hash', 'y-up', 'y-down', 'y-left', 'y-right', 'line-ew', 'line-ns', 'line-ne', 'line-nw']


for arr in arrs:
    assert arr.shape[0] == n_steps
    mean_trajs.append(arr)

if steplist is not None: 
    steps = steplist
else: 
    steps = list(range(n_steps))

# Create Plotly traces
traces = []
lidx, cidx, midx = 0, 1, 2
labelsL = {label: idx for idx, label in enumerate(list(set([label.split(":")[lidx] for label in labels])))}
labelsC = {label: idx for idx, label in enumerate(list(set([label.split(":")[cidx] for label in labels])))}
labelsM = {label: idx for idx, label in enumerate(list(set([label.split(":")[midx] for label in labels])))}
for idx, (mean_traj, label) in enumerate(zip(mean_trajs, labels)):
    # Add mean line
    traces.append(go.Scatter(
        x=steps,
        y=mean_traj,
        mode="lines+markers", 
        name=label,
        line=dict(color=colors[labelsC[label.split(":")[cidx]]], dash=lines[labelsL[label.split(":")[lidx]]]),
        marker=dict(symbol=marks[labelsM[label.split(":")[midx]]]), 
        visible=True
    ))

fig = go.Figure(data=traces)

fieldlist = fieldlist if fieldlist is not None else range(len(re.split(r'[:;]', labels[0])))
fieldedlabels = {field: list(set([re.split(r'[:;]', label)[fidx] for label in labels])) for fidx, field in enumerate(fieldlist)}

checkbox_list = []
for fidx, field in enumerate(fieldlist): 
    buttonlist = []
    all_signal_setter = [-1 if idx == fidx else None for idx in range(len(fieldlist))]
    buttonlist.append(dict(label=f'{field}:All', method='restyle', args=[{'visible': getset_visibility(all_signal_setter)}]))
    for fieldlabel in fieldedlabels[field]: 
        field_signal_setter = [fieldlabel if idx == fidx else None for idx in range(len(fieldlist))]
        buttonlist.append(dict(label=f"{field}:{fieldlabel}", method='restyle', args=[{'visible': getset_visibility(field_signal_setter)}]))
    checkbox_dict = dict(
        buttons=buttonlist,
        type="dropdown", 
        direction="down",
        showactive=True,
        x=0.1 + 0.2 * fidx,
        xanchor="left",
        y=1.15,
        yanchor="top"
    )
    checkbox_list.append(checkbox_dict)

# Update layout
fig.update_layout(
    title=plot_label_dict["title"],
    xaxis_title=plot_label_dict["xlabel"],
    yaxis_title=plot_label_dict["ylabel"],
    yaxis=dict(range=y_range if y_range is not None else [None, None]),
    legend_title="Trajectories",
    template="plotly_white"
)

fig.update_layout(
    updatemenus=checkbox_list,
    showlegend=True
)
field_select_signal = [None for _ in range(len(fieldlist))]
# Save to HTML file
fig.write_html(save_path)

In [157]:
plot_many_plotly_meaned_menu(data_list, tag_list, os.path.join(comp_plots_dir, f"hidrep-attnout-ori.html"), 
                plot_label_dict={"xlabel": "Epoch", "ylabel": "Scores Value", "title": f"Comparison Trajectories of PSAS and Layer Outputs"}, 
                cloud=False, fieldlist=["Cond", "Eval", "Rep", "Dim"])

## Quantify

In [87]:
from sklearn.preprocessing import StandardScaler

In [93]:
allcorrres = {}

### Pearson's Correlation

In [124]:
from scipy.stats import pearsonr

In [125]:
allcorrres["pearsonr"] = {"PSAS-hidrep": [], "PSAS-attnout": [], "PSAS-encrnn5f": []}
for hiddim in [4, 8, 16, 32, 48, 64]: 
    attnres = super_data_collector[hiddim]["attnres"]["ST"][1]
    hidrepres = super_data_collector[hiddim]["layered_res"]["hidrep"]
    attnoutres = super_data_collector[hiddim]["layered_res"]["attnout"]
    encrnn5fres = super_data_collector[hiddim]["layered_res"]["enc-rnn5-f"]

    meaned_attnres = np.mean(attnres, axis=1)
    meaned_hidrepres = np.mean(hidrepres, axis=1)
    meaned_attnoutres = np.mean(attnoutres, axis=1)
    meaned_encrnn5fres = np.mean(encrnn5fres, axis=1)

    scaler = StandardScaler()
    meaned_attnres_norm = scaler.fit_transform(meaned_attnres.reshape(-1, 1)).flatten()
    meaned_hidrepres_norm = scaler.fit_transform(meaned_hidrepres.reshape(-1, 1)).flatten()
    meaned_attnoutres_norm = scaler.fit_transform(meaned_attnoutres.reshape(-1, 1)).flatten()
    meaned_encrnn5fres_norm = scaler.fit_transform(meaned_encrnn5fres.reshape(-1, 1)).flatten()

    allcorrres["pearsonr"]["PSAS-hidrep"].append(pearsonr(meaned_attnres_norm, meaned_hidrepres_norm)[0])
    allcorrres["pearsonr"]["PSAS-attnout"].append(pearsonr(meaned_attnres_norm, meaned_attnoutres_norm)[0])
    allcorrres["pearsonr"]["PSAS-encrnn5f"].append(pearsonr(meaned_attnres_norm, meaned_encrnn5fres_norm)[0])

In [126]:
allcorrres["pearsonr"]

{'PSAS-hidrep': [0.311091427393406,
  -0.0777652361673118,
  0.03437726085834591,
  0.775068545773404,
  0.7573090637809073,
  0.7839572296183972],
 'PSAS-attnout': [0.30251421341802537,
  -0.04798396944702417,
  -0.4030583730215058,
  -0.492742607548129,
  -0.3516524917151794,
  -0.5658207080111815],
 'PSAS-encrnn5f': [-0.17936565510915625,
  -0.13188934565377794,
  0.6006775853105506,
  0.898717019034735,
  0.862299999025164,
  0.8746293206096171]}

In [127]:
allcorrres

{'pearsonr': {'PSAS-hidrep': [0.311091427393406,
   -0.0777652361673118,
   0.03437726085834591,
   0.775068545773404,
   0.7573090637809073,
   0.7839572296183972],
  'PSAS-attnout': [0.30251421341802537,
   -0.04798396944702417,
   -0.4030583730215058,
   -0.492742607548129,
   -0.3516524917151794,
   -0.5658207080111815],
  'PSAS-encrnn5f': [-0.17936565510915625,
   -0.13188934565377794,
   0.6006775853105506,
   0.898717019034735,
   0.862299999025164,
   0.8746293206096171]},
 'spearman': {'PSAS-hidrep': [0.2192195690157251,
   -0.3531974373907979,
   0.22087361677344203,
   0.773185789167152,
   0.7948165404775771,
   0.5551310425160163],
  'PSAS-attnout': [-0.01700640652300524,
   -0.21224228305183457,
   -0.40952824694234125,
   -0.5653581828771111,
   -0.04920209668025626,
   -0.741199767035527],
  'PSAS-encrnn5f': [0.23813628421665692,
   -0.3398252766453116,
   0.6566802562609201,
   0.8623762376237623,
   0.6260920209668025,
   0.33761211415259174]},
 'euclidean': {'PSAS-hi

### Spearman's Correlation

In [128]:
from scipy.stats import spearmanr

In [129]:
allcorrres["spearman"] = {"PSAS-hidrep": [], "PSAS-attnout": [], "PSAS-encrnn5f": []}
for hiddim in [4, 8, 16, 32, 48, 64]: 
    attnres = super_data_collector[hiddim]["attnres"]["ST"][1]
    hidrepres = super_data_collector[hiddim]["layered_res"]["hidrep"]
    attnoutres = super_data_collector[hiddim]["layered_res"]["attnout"]
    encrnn5fres = super_data_collector[hiddim]["layered_res"]["enc-rnn5-f"]

    meaned_attnres = np.mean(attnres, axis=1)
    meaned_hidrepres = np.mean(hidrepres, axis=1)
    meaned_attnoutres = np.mean(attnoutres, axis=1)
    meaned_encrnn5fres = np.mean(encrnn5fres, axis=1)

    scaler = StandardScaler()
    meaned_attnres_norm = scaler.fit_transform(meaned_attnres.reshape(-1, 1)).flatten()
    meaned_hidrepres_norm = scaler.fit_transform(meaned_hidrepres.reshape(-1, 1)).flatten()
    meaned_attnoutres_norm = scaler.fit_transform(meaned_attnoutres.reshape(-1, 1)).flatten()
    meaned_encrnn5fres_norm = scaler.fit_transform(meaned_encrnn5fres.reshape(-1, 1)).flatten()

    allcorrres["spearman"]["PSAS-hidrep"].append(spearmanr(meaned_attnres_norm, meaned_hidrepres_norm)[0])
    allcorrres["spearman"]["PSAS-attnout"].append(spearmanr(meaned_attnres_norm, meaned_attnoutres_norm)[0])
    allcorrres["spearman"]["PSAS-encrnn5f"].append(spearmanr(meaned_attnres_norm, meaned_encrnn5fres_norm)[0])

In [130]:
allcorrres["spearman"]

{'PSAS-hidrep': [0.2192195690157251,
  -0.3531974373907979,
  0.22087361677344203,
  0.773185789167152,
  0.7948165404775771,
  0.5551310425160163],
 'PSAS-attnout': [-0.01700640652300524,
  -0.21224228305183457,
  -0.40952824694234125,
  -0.5653581828771111,
  -0.04920209668025626,
  -0.741199767035527],
 'PSAS-encrnn5f': [0.23813628421665692,
  -0.3398252766453116,
  0.6566802562609201,
  0.8623762376237623,
  0.6260920209668025,
  0.33761211415259174]}

### DTW

In [131]:
from dtaidistance import dtw
from dtaidistance import dtw_visualisation as dtwvis
import matplotlib.pyplot as plt

In [132]:
allcorrres["dtw"] = {"PSAS-hidrep": [], "PSAS-attnout": [], "PSAS-encrnn5f": []}
for hiddim in [4, 8, 16, 32, 48, 64]: 
    attnres = super_data_collector[hiddim]["attnres"]["ST"][1]
    hidrepres = super_data_collector[hiddim]["layered_res"]["hidrep"]
    attnoutres = super_data_collector[hiddim]["layered_res"]["attnout"]
    encrnn5fres = super_data_collector[hiddim]["layered_res"]["enc-rnn5-f"]

    meaned_attnres = np.mean(attnres, axis=1)
    meaned_hidrepres = np.mean(hidrepres, axis=1)
    meaned_attnoutres = np.mean(attnoutres, axis=1)
    meaned_encrnn5fres = np.mean(encrnn5fres, axis=1)

    scaler = StandardScaler()
    meaned_attnres_norm = scaler.fit_transform(meaned_attnres.reshape(-1, 1)).flatten()
    meaned_hidrepres_norm = scaler.fit_transform(meaned_hidrepres.reshape(-1, 1)).flatten()
    meaned_attnoutres_norm = scaler.fit_transform(meaned_attnoutres.reshape(-1, 1)).flatten()
    meaned_encrnn5fres_norm = scaler.fit_transform(meaned_encrnn5fres.reshape(-1, 1)).flatten()

    dtw_distance_norm_PSAS_hidrep = dtw.distance(meaned_attnres_norm, meaned_hidrepres_norm)
    dtw_distance_norm_PSAS_attnout = dtw.distance(meaned_attnres_norm, meaned_attnoutres_norm)
    dtw_distance_norm_PSAS_encrnn5f = dtw.distance(meaned_attnres_norm, meaned_encrnn5fres_norm)

    allcorrres["dtw"]["PSAS-hidrep"].append(dtw_distance_norm_PSAS_hidrep)
    allcorrres["dtw"]["PSAS-attnout"].append(dtw_distance_norm_PSAS_attnout)
    allcorrres["dtw"]["PSAS-encrnn5f"].append(dtw_distance_norm_PSAS_encrnn5f)



    # # mean_trajectory1, mean_trajectory2 = meaned_attnres, meaned_hidrepres
    # dtw_distance = dtw.distance(mean_trajectory1_norm, mean_trajectory2_norm)
    # # print(f"PSAS-hidrep: {hiddim}, DTW distance: {dtw_distance}")
    # alignment = dtw.warping_path(mean_trajectory1_norm, mean_trajectory2_norm)
    # dtwvis.plot_warping(mean_trajectory1_norm, mean_trajectory2_norm, alignment)
    # plt.show()

In [133]:
allcorrres["dtw"]

{'PSAS-hidrep': [7.865334376152704,
  6.195000288852409,
  3.9969693589934843,
  3.2365043875942345,
  4.087174267874087,
  4.2876932862325345],
 'PSAS-attnout': [8.205413009727234,
  8.036794283179782,
  9.76463294791076,
  12.717805258955964,
  12.570357653364061,
  16.47757769422475],
 'PSAS-encrnn5f': [13.322447496730312,
  6.086689611368907,
  3.001827728450881,
  1.6580312817329712,
  2.349606170593076,
  3.035367739478126]}

### Euclidean Distance

In [134]:
from scipy.spatial.distance import euclidean

In [135]:
allcorrres["euclidean"] = {"PSAS-hidrep": [], "PSAS-attnout": [], "PSAS-encrnn5f": []}
for hiddim in [4, 8, 16, 32, 48, 64]: 
    attnres = super_data_collector[hiddim]["attnres"]["ST"][1]
    hidrepres = super_data_collector[hiddim]["layered_res"]["hidrep"]
    attnoutres = super_data_collector[hiddim]["layered_res"]["attnout"]
    encrnn5fres = super_data_collector[hiddim]["layered_res"]["enc-rnn5-f"]

    meaned_attnres = np.mean(attnres, axis=1)
    meaned_hidrepres = np.mean(hidrepres, axis=1)
    meaned_attnoutres = np.mean(attnoutres, axis=1)
    meaned_encrnn5fres = np.mean(encrnn5fres, axis=1)

    scaler = StandardScaler()
    meaned_attnres_norm = scaler.fit_transform(meaned_attnres.reshape(-1, 1)).flatten()
    meaned_hidrepres_norm = scaler.fit_transform(meaned_hidrepres.reshape(-1, 1)).flatten()
    meaned_attnoutres_norm = scaler.fit_transform(meaned_attnoutres.reshape(-1, 1)).flatten()
    meaned_encrnn5fres_norm = scaler.fit_transform(meaned_encrnn5fres.reshape(-1, 1)).flatten()

    allcorrres["euclidean"]["PSAS-hidrep"].append(euclidean(meaned_attnres_norm, meaned_hidrepres_norm))
    allcorrres["euclidean"]["PSAS-attnout"].append(euclidean(meaned_attnres_norm, meaned_attnoutres_norm))
    allcorrres["euclidean"]["PSAS-encrnn5f"].append(euclidean(meaned_attnres_norm, meaned_encrnn5fres_norm))

In [136]:
allcorrres["euclidean"]

{'PSAS-hidrep': [11.79659005557959,
  14.754950844680547,
  13.96623761945797,
  6.740634478407835,
  7.001683196697918,
  6.606106281410758],
 'PSAS-attnout': [11.86979924969566,
  14.549665241578191,
  16.83501681787545,
  17.364734459833038,
  16.523734287652065,
  17.784706555140687],
 'PSAS-encrnn5f': [15.434761784972501,
  15.120900911673395,
  8.981265375270308,
  4.523180506324717,
  5.274030653038556,
  5.032382892415659]}

In [138]:
allcorrres

{'pearsonr': {'PSAS-hidrep': [0.311091427393406,
   -0.0777652361673118,
   0.03437726085834591,
   0.775068545773404,
   0.7573090637809073,
   0.7839572296183972],
  'PSAS-attnout': [0.30251421341802537,
   -0.04798396944702417,
   -0.4030583730215058,
   -0.492742607548129,
   -0.3516524917151794,
   -0.5658207080111815],
  'PSAS-encrnn5f': [-0.17936565510915625,
   -0.13188934565377794,
   0.6006775853105506,
   0.898717019034735,
   0.862299999025164,
   0.8746293206096171]},
 'spearman': {'PSAS-hidrep': [0.2192195690157251,
   -0.3531974373907979,
   0.22087361677344203,
   0.773185789167152,
   0.7948165404775771,
   0.5551310425160163],
  'PSAS-attnout': [-0.01700640652300524,
   -0.21224228305183457,
   -0.40952824694234125,
   -0.5653581828771111,
   -0.04920209668025626,
   -0.741199767035527],
  'PSAS-encrnn5f': [0.23813628421665692,
   -0.3398252766453116,
   0.6566802562609201,
   0.8623762376237623,
   0.6260920209668025,
   0.33761211415259174]},
 'euclidean': {'PSAS-hi

# Plot all Results

In [148]:
data_list = []
tag_list = []
for metric in allcorrres.keys(): 
    for target in allcorrres[metric].keys(): 
        data_list.append(np.array(allcorrres[metric][target]).reshape(1, -1))
        tag_list.append(f"{metric}:{target}")

In [151]:
plot_many_plotly(data_list, tag_list, os.path.join(comp_plots_dir, f"Metrics.html"), 
                plot_label_dict={"xlabel": "Dimension", "ylabel": "Scores Value", "title": f"Matching Metrics Comparison"}, 
                cloud=False, steplist=[4, 8, 16, 32, 48, 64])

/home/franklhtan/anaconda3/envs/wavln/lib/python3.11/site-packages/numpy/core/_methods.py:206: RuntimeWarning:

Degrees of freedom <= 0 for slice

/home/franklhtan/anaconda3/envs/wavln/lib/python3.11/site-packages/numpy/core/_methods.py:195: RuntimeWarning:

invalid value encountered in divide



In [74]:
with open(os.path.join(comp_plots_dir, "alldata.pkl"), "bw") as f: 
    pickle.dump(super_data_collector, f)